In [13]:
from vizHandObj.hand_robot_viewer import RobotHandViewer
from vizHandObj import ThreeJawGripperViewer
import numpy as np

import json
import pickle
from pathlib import Path
from typing import Dict, List, Optional
import sapien
import numpy as np
import torch
import tyro
from pytransform3d import rotations
from tqdm import tqdm

from vizHandObj.dataset import DexYCBVideoDataset
from dex_retargeting.constants import (
    HandType,
    RetargetingType,
    RobotName,
    get_default_config_path,
    ROBOT_NAME_MAP,
)
from dex_retargeting.retargeting_config import RetargetingConfig
from dex_retargeting.seq_retarget import SeqRetargeting
from vizHandObj.mano_layer import MANOLayer
from pytransform3d import transformations as pt


# Create Dataset From DEXYCB

In [14]:
robot_dir = "/home/main/dex-ICLR/dex-retargeting/assets/robots/hands"
dexycb_dir = "/home/main/data/dexycb"
output_dir = "/home/main/data/robot_data"

In [3]:
YCB_CLASSES = {
    1: "002_master_chef_can",
    2: "003_cracker_box",
    3: "004_sugar_box",
    4: "005_tomato_soup_can",
    5: "006_mustard_bottle",
    6: "007_tuna_fish_can",
    7: "008_pudding_box",
    8: "009_gelatin_box",
    9: "010_potted_meat_can",
    10: "011_banana",
    11: "019_pitcher_base",
    12: "021_bleach_cleanser",
    13: "024_bowl",
    14: "025_mug",
    15: "035_power_drill",
    16: "036_wood_block",
    17: "037_scissors",
    18: "040_large_marker",
    19: "051_large_clamp",
    20: "052_extra_large_clamp",
    21: "061_foam_brick",
}

In [4]:

RetargetingConfig.set_default_urdf_dir(robot_dir)

if not Path(dexycb_dir).exists():
        raise ValueError(f"DexYCB dir not exists: {dexycb_dir}")

viewer = RobotHandViewer(robot_names=[RobotName.allegro, 
                                      RobotName.shadow,
                                      RobotName.svh,
                                      RobotName.leap,
                                      RobotName.ability,
                                      RobotName.panda,
                                      ],
                          hand_type= HandType.right,
                          headless=True,
                          )
gripper_viewer = ThreeJawGripperViewer(urdf_path="/home/main/dex-ICLR/dex-retargeting/assets/robots/hands/dclaw_gripper/dclaw_gripper_glb.urdf", headless=True)
dataset = DexYCBVideoDataset(
    dexycb_dir, 
    hand_type="right",
    filter_objects=[]
)

success_count = 0
total_count = len(dataset)
failed_captures = []

for i in tqdm(range(total_count), desc="parse traj..."):
    try:
        data = dataset[i]
        capture_name = data["capture_name"]
        viewer.load_object_hand(data)
        result = viewer.retargeting_only(data)
        gripper_viewer.load_object_hand(data)
        gripper_result = gripper_viewer.retargeting_only(data)
        item = {
                    "robot_name": gripper_result["robot_name"],
                    "robot_qpos": gripper_result["robot_qpos"],
            }
        result["gripper"] = item
        ycb_ids_names = [" ".join(YCB_CLASSES[ycb_id].split("_")[1:]) for ycb_id in data["ycb_ids"]]
        result["ycb_ids_names"] = ycb_ids_names
        result["grasped_ycb_id"] = data["ycb_ids"][np.argmax(np.sum(data["object_pose"][0]-data["object_pose"][-1]>0,axis=1))]
        result["grasped_ycb_name"] = ycb_ids_names[np.argmax(np.sum(data["object_pose"][0]-data["object_pose"][-1]>0,axis=1))]

        if result is not None:
            output_file = f"{output_dir}/{capture_name}"
            np.savez_compressed(
                f"{output_file}.npz",
                **result
            )
        else:
            failed_captures.append(capture_name)
    except Exception as e:
        print(f"Failed to process {capture_name}: {e}")

parse traj...:   0%|          | 0/501 [00:00<?, ?it/s]/home/main/dex-ICLR/dex-retargeting/example/position_retargeting/manopth/manopth/manolayer.py:72: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:203.)
  torch.Tensor(smpl_data['betas'].r).unsqueeze(0))
/home/main/dex-ICLR/dex-retargeting/src/dex_retargeting/robot_wrapper.py:69: UserWarning: Deprecated member. Use Frame.parentJoint instead.
  parent_id = self.model.frames[joint_id].parent
/home/main/dex-ICLR/dex-retargeting/src/dex_retargeting/robot_wrapper.py:72: UserWarning: Deprecated member. Use Frame.parentFrame instead.
  if frame.previousFrame == joint_id:
parse tra

Failed to process 20201002_111900: 'NoneType' object is not subscriptable


parse traj...: 100%|██████████| 501/501 [1:02:29<00:00,  7.48s/it]


# Create Dataset From OAKINK

In [39]:
output_dir = "/home/main/data/oakink_robot_data"

In [40]:
import numpy as np
from glob import glob
import os 
RetargetingConfig.set_default_urdf_dir(robot_dir)
files = glob(f"/home/main/data/oakink/extracted_sequences/*.pkl")
viewer = RobotHandViewer(robot_names=[RobotName.allegro, 
                                    #   RobotName.shadow,
                                    #   RobotName.svh,
                                    #   RobotName.leap,
                                    #   RobotName.ability,
                                    #   RobotName.panda,
                                      ],
                          hand_type= HandType.right,
                          headless=True,
                          )
gripper_viewer = ThreeJawGripperViewer(urdf_path="/home/main/dex-ICLR/dex-retargeting/assets/robots/hands/dclaw_gripper/dclaw_gripper_glb.urdf", headless=True)

for i in tqdm(range(len(files))):
    try:
        data = np.load(files[i],allow_pickle=True)
        capture_name = data["seq_id"]
        # 替换mesh的跟路径
        mesh_root = "/home/main/data/oakink/obj"
        mesh_path = os.path.join(mesh_root, os.path.basename(data["object_mesh_path_list"][0]))
        data_load_obj = {
        "ycb_ids": [1],
        "object_mesh_file":mesh_path,
        "hand_shape":np.array(data["mano_shape"]),
        "extrinsics":np.identity(4)
        }
        viewer.load_object_hand(data_load_obj)
        data_ = {
        "ycb_ids":[1],
        "object_mesh_file":mesh_path,
        "object_pose":data["cam0_object_pose_seq_world"],
        "hand_shape":np.array(data["mano_shape"]),
        "extrinsics":np.identity(4),
        "hand_pose":np.expand_dims(np.array(data["mano_pose_seq"]),axis=1),
        "capture_name": data["seq_id"],
        "start_frame": 0
        }
        result = viewer.retargeting_only(data_)
        gripper_viewer.load_object_hand(data_load_obj)
        gripper_result = gripper_viewer.retargeting_only(data_)
        item = {
                    "robot_name": gripper_result["robot_name"],
                    "robot_qpos": gripper_result["robot_qpos"],
            }
        result["gripper"] = item
        ycb_ids_names = [1]
        result["ycb_ids_names"] = ycb_ids_names
        result["grasped_ycb_id"] = data["object_id"]
        result["grasped_ycb_name"] = data["object_id"]

        if result is not None:
            output_file = f"{output_dir}/{capture_name}"
            np.savez_compressed(
                f"{output_file}.npz",
                **result
            )
    except Exception as e:
        print(f"Failed to process {capture_name}: {e}")
    # viewer.render_dexycb_data(data_,
    #                       video_path="/home/main/dex-ICLR/UniHM/videos/hands.mp4",
    #                     #   precomputed_qpos = pre_pose,

    #                       )

  0%|          | 0/988 [00:00<?, ?it/s]/home/main/dex-ICLR/dex-retargeting/src/dex_retargeting/robot_wrapper.py:69: UserWarning: Deprecated member. Use Frame.parentJoint instead.
  parent_id = self.model.frames[joint_id].parent
/home/main/dex-ICLR/dex-retargeting/src/dex_retargeting/robot_wrapper.py:72: UserWarning: Deprecated member. Use Frame.parentFrame instead.
  if frame.previousFrame == joint_id:
  0%|          | 1/988 [00:21<5:53:07, 21.47s/it]


KeyboardInterrupt: 

In [30]:
result

{'capture_name': 'A02011_0004_0000_0004',
 'object_pose': array([[[ 0.00834097,  0.00566399,  0.19270831, ...,  0.0284481 ,
          -0.01867178,  0.11681625]],
 
        [[ 0.00847136,  0.00502712,  0.19293916, ...,  0.02836561,
          -0.01870748,  0.11678648]],
 
        [[ 0.00838297,  0.00575508,  0.19279543, ...,  0.02845842,
          -0.0186626 ,  0.11679214]],
 
        ...,
 
        [[ 0.04777733, -0.17138235,  0.13389769, ...,  0.053011  ,
           0.24288738,  0.23807648]],
 
        [[ 0.02023078, -0.08308228,  0.11739643, ...,  0.03998196,
           0.23836279,  0.2224445 ]],
 
        [[ 0.00764776, -0.04122255,  0.10918565, ...,  0.0321101 ,
           0.23598307,  0.21330813]]], shape=(164, 1, 7), dtype=float32),
 'extrinsics': array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]),
 'ycb_ids': [1],
 'object_mesh_file': '/home/main/data/oakink/obj/A02011.obj',
 'hand_shape': array([ 5.4091064e-04,  3.9712257e-05,

In [6]:
data.keys()

dict_keys(['seq_id', 'timestamp', 'intent_id', 'intent_name', 'subject_flag', 'object_id', 'object_id_list', 'object_mesh_path_list', 'cam_ids', 'object_pose_order', 'mano_pose_seq', 'mano_shape', 'cam0_object_pose_seq', 'cam0_object_pose_seq_world', 'cam0_cam_extr', 'cam1_object_pose_seq', 'cam1_object_pose_seq_world', 'cam1_cam_extr', 'cam2_object_pose_seq', 'cam2_object_pose_seq_world', 'cam2_cam_extr', 'cam3_object_pose_seq', 'cam3_object_pose_seq_world', 'cam3_cam_extr'])

In [38]:
data["seq_id"]

'A02011_0004_0000_0004'

In [17]:
np.array(data["mano_shape"])

array([ 5.4091064e-04,  3.9712257e-05,  7.5765580e-05,  1.8573153e-05,
       -2.3872378e-06, -1.9745655e-06, -3.6045913e-05,  1.5522059e-05,
       -7.3576252e-06,  1.0333585e-05], dtype=float32)

In [11]:
os.path.join("/home/main/data/oakink/obj", os.path.basename(data["object_mesh_path_list"][0]))

'/home/main/data/oakink/obj/A02011.obj'

# Create Dataset For VQVAE

In [3]:
import numpy as np
from glob import glob

In [ ]:
robot_dir = "/path/to/data/robot_data"
output_dir = "/path/to/data/robot_data/dataset.npz"

In [15]:
npzfiles = glob(f"{robot_dir}/*.npz")
result = []
for npzfile in npzfiles:
    data = np.load(npzfile, allow_pickle=True)

    for i in range(len(data["hand_pose"])):
        sample ={
            "hand_pose": data["hand_pose"][i],
            "allegro_hand_qpos": dict(data["allegro_hand"].tolist())["robot_qpos"][i],
            "shadow_hand_qpos": dict(data["shadow_hand"].tolist())["robot_qpos"][i],
            "svh_hand_qpos": dict(data["schunk_svh_hand"].tolist())["robot_qpos"][i],
            "leap_hand_qpos": dict(data["leap_hand"].tolist())["robot_qpos"][i],
            "ability_hand_qpos": dict(data["ability_hand"].tolist())["robot_qpos"][i],
            "panda_hand_qpos": dict(data["panda_gripper"].tolist())["robot_qpos"][i],
        }
        result.append(sample)

In [18]:
result = np.array(result, dtype=object)
np.savez_compressed(output_dir, data=result)